In [2]:
import streamlit as st
import pandas as pd
from datetime import datetime
import os
from glob import glob
import matplotlib.pyplot as plt

st.set_page_config(
    page_title="Wide Page Example",  # Optional: Set a custom page title
    layout="wide",  # Use wide layout
    initial_sidebar_state="expanded"  # Keep the sidebar expanded
)

# Title and Logo
st.write("""
# Databases Latest Backup's
You can check here the Database's Name With Their Latest **Backup Date**
""")

#--------------------------------------------#

# List of main directories to search for backup files (network paths)
main_directories = [
    r"\\192.168.43.146\Backups\Daily_Full_Backups", 
    r"\\192.168.43.146\Backups\Daily_Partial_Backups"
]

# Dictionary to store the latest files for each folder
latest_files = {}

# Iterate through each main directory
for main_dir in main_directories:
    try:
        # Log the directory being scanned
        st.write(f"Scanning directory: {main_dir}")  # Debugging log
        
        # Check if the directory is accessible
        if not os.path.exists(main_dir):
            st.warning(f"Directory {main_dir} is not accessible or does not exist.")
            continue
        
        # List folders in the main directory
        for folder in os.listdir(main_dir):
            folder_path = os.path.join(main_dir, folder)
            
            # Check if the path is a directory
            if os.path.isdir(folder_path):
                # Find all .bak files in the directory
                files = glob(os.path.join(folder_path, '*.bak'))
                
                if files:  # If .bak files are found
                    latest_file = max(files, key=os.path.getmtime)  # Find the latest file
                    latest_files[folder + str(latest_file.title)] = latest_file
                    
    except Exception as e:
        st.warning(f"Error accessing directory {main_dir}: {e}")  # Display any errors

# Create a DataFrame for display
data = {
    "Folder": [],
    "Latest Backup File": [],
    "Last Modified": [],
    "Path": [],  # Add the 'Path' column to show the directory (network path)
    "Days Passed": [],
    "Title" : [],
    "Differential Hours" : []
}

for folder, latest_file in latest_files.items():
    modified_time = datetime.fromtimestamp(os.path.getmtime(latest_file))
    # Determine which directory the backup file is from
    path = "Daily_Partial_Backups" if latest_file.startswith(r"\\192.168.43.146\Backups\Daily_Partial_Backups") else "Daily_Full_Backups"
    data["Folder"].append(folder.split("<")[0])
    data["Latest Backup File"].append(latest_file)
    data["Last Modified"].append(modified_time)
    data["Path"].append(path)
    data["Title"].append(folder.split("<")[0]+ str(" ") +str(path))
    days_diff = (datetime.today() - modified_time).days
    data["Days Passed"].append(days_diff)
    
    path_hours = round((datetime.now() - modified_time).total_seconds() / 3600, 2) if path == "Daily_Partial_Backups" else 0  # Leave empty for Daily_Full_Backups
    data["Differential Hours"].append(path_hours)

df = pd.DataFrame(data)

# Sidebar for user interaction
st.sidebar.header("User Input Features")

# Add a select box for single selection
selected_db_single = st.sidebar.selectbox(
    "Select a Single Database",
    options=df["Folder"].tolist(),
    help="Choose one database folder to view its latest backup details."
)

# Add a multi-select for multiple selections
selected_dbs_multi = st.sidebar.multiselect(
    "Select Database(s)",
    options=df["Folder"].tolist(),
    default=df["Folder"].tolist(),
    help="Choose one or more database folders to view their backup details."
)

# Add a search bar to filter databases
search_query = st.sidebar.text_input("Search Databases", "", help="Type to search for specific database folders.")
if search_query:
    df = df[df["Folder"].str.contains(search_query, case=False, na=False)]

# Filter DataFrame based on user selection
filtered_df_single = df[df["Folder"] == selected_db_single]
filtered_df_multi = df[df["Folder"].isin(selected_dbs_multi)]

# Display filtered results
st.markdown("### Selected Database Information (Single Selection)", help="Details for the single database folder selected.")
st.write(filtered_df_single)

st.markdown("### Latest Backup Information (Multi-Selection)", help="Details for all database folders selected.")
st.write(filtered_df_multi)

# Visualization for full backups
st.title("Database Backup Summary")
st.header("Full Backups")
st.text("Full Backups Took Place after 3 Days: {lenf(df)}")
if not df.empty:
    selected_df = df[["Title", "Days Passed"]]
    color = "#9a031e" if selected_df["Title"].str.contains("Full").any() else "#0a9396"
    st.bar_chart(selected_df.set_index("Title"), color=color)
    #plt.axhline(y=3, color="r", linestyle="-")  
    #st.pyplot(plt)
else:
    st.info("No databases match your search query.")

# Visualization for Differential backups
st.header("Partial Backups")
st.text("Partial Backups Took Place after every Four Hours")
if not df.empty:
    partial_df = df[["Title", "Differential Hours"]]
    color = "#5f0f40" if partial_df["Title"].str.contains("Partial").any() else "#43291f"
    st.bar_chart(partial_df.set_index("Title"), color=color)
else:
    st.info("No Database math you search query")
    
    
# Additional feature: Display total backups found
st.markdown(f"##### Total Backups Found: {len(df)}")


Latest backup file: D:/CheckBackStatus\adv_workdw_fullbak19_20241114.bak
Last modified: 2024-11-14 10:24:11.081335
Latest backup file: D:/FarazGill\adv_work_fullbak19_20241114.bak
Last modified: 2024-11-14 10:26:31.986824
Latest backup file: E:/Backups\adv_work_fullbak19_20241114.bak
Last modified: 2024-11-14 10:26:31.986824
